In [4]:
#사무실에서 바로 이거 쓰면 됨 /

import subprocess
import pyautogui
import time as tm
import cv2
import numpy as np
import mss
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# 작업 디렉토리 설정
os.chdir("C:/Users/USER/Desktop/이체매크로")

print("Current working directory:", os.getcwd())

# 이미지 파일 경로 설정
COORDS = {
    "localdisktab": "localdisk.png",
    "outdisktab": "seagate_bt.png",
    "commonlogin": "localdisk_loginbt.png",
    "send_tab": "send_tab.png",
    "multisend": "multisend_bt.png",
    "password4_input": "password4_input.png",
}

monitor_1 = {'top': 0, 'left': 1920, 'width': 1920, 'height': 1080}
monitor_2 = {'top': 0, 'left': 0, 'width': 1920, 'height': 1080}

def capture_screen(region):
    with mss.mss() as sct:
        screenshot = sct.grab(region)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        return img

def match_template(screen, template_path):
    template = cv2.imread(template_path, cv2.IMREAD_COLOR)
    if template is None:
        raise ValueError(f"이미지를 찾을 수 없습니다: {template_path}")

    result = cv2.matchTemplate(screen, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)
    if max_val >= 0.9:  # 일치율이 90% 이상인 경우
        return (max_loc[0] + template.shape[1] // 2, max_loc[1] + template.shape[0] // 2)
    return None

def locate_image_on_monitor(image_path, monitor):
    screen = capture_screen(monitor)
    location = match_template(screen, image_path)
    if location:
        location = (location[0] + monitor['left'], location[1] + monitor['top'])
    return location

def locate_image_on_monitors(image_path):
    location = locate_image_on_monitor(image_path, monitor_1)
    if not location:
        location = locate_image_on_monitor(image_path, monitor_2)
    return location

def locate_and_click(image_path):
    print(f"{image_path}을(를) 찾고 클릭합니다.")
    location = locate_image_on_monitors(image_path)
    if location:
        print(f"{image_path}의 위치를 찾았습니다: {location}, 클릭합니다.")
        pyautogui.click(location)
        tm.sleep(0.5)
    else:
        print(f"{image_path}의 위치를 찾을 수 없습니다.")

def scroll_down():
    pyautogui.scroll(-10000)


# Selenium 설정 및 웹페이지 열기
url = 'https://www.kebhana.com/common/login.do'
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url)

tm.sleep(5)

print(driver.find_element(By.CSS_SELECTOR, "#certLogin").text) 
driver.find_element(By.CSS_SELECTOR, "#certLogin").click()

tm.sleep(7)

# PyAutoGUI를 이용한 인증서 로그인
def login_with_certificate():
    locate_and_click(COORDS["localdisktab"])   # 로컬디스크
    tm.sleep(2)
    locate_and_click(COORDS["outdisktab"])     # 외장하드선택
    tm.sleep(3)
    
    for _ in range(6):   # 탭 6번
        pyautogui.press('tab')
        tm.sleep(0.1)
    
    # 비밀번호 입력 (한글자씩 입력)
    password = "!gusqls120"
    for char in password:
        pyautogui.press(char)
        tm.sleep(0.01)
    
    locate_and_click(COORDS["commonlogin"])    # 공동로그인 버튼 클릭

login_with_certificate()
tm.sleep(8)

# 프레임 전환
frame_id = "hanaMainframe"  # 프레임 ID를 여기에 입력하세요
driver.switch_to.frame(frame_id)

# 이체 탭 클릭
locate_and_click(COORDS["send_tab"])
tm.sleep(1)

# 다계좌 이체 버튼 클릭
for _ in range(3):   # 탭 3번
    pyautogui.press('tab')

pyautogui.press('enter')   
tm.sleep(3)

def scroll_down(amount):
    pyautogui.scroll(-amount)  # 스크롤 다운

def scroll_up(amount):
    pyautogui.scroll(amount)  # 스크롤 업

# 예시: 전체 페이지를 스크롤 다운하고 절반만 다시 스크롤 업
scroll_amount = 1000  # 페이지의 길이에 맞게 조정
scroll_down(scroll_amount)
tm.sleep(1)  # 잠시 대기
scroll_up(scroll_amount // 2)

# 은행 선택을 위한 맵핑
bank_options = {
    "하나은행": "081",
    "경남은행": "039",
    "광주은행": "034",
    "국민은행": "004",
    "기업은행": "003",
    "농협": "011",
    "iM뱅크(대구)": "031",
    "도이치뱅크": "055",
    "부산은행": "032",
    "산업은행": "002",
    "저축은행": "050",
    "새마을금고": "045",
    "수협은행": "007",
    "신협": "048",
    "신한은행": "088",
    "우리은행": "020",
    "우체국": "071",
    "전북은행": "037",
    "제주은행": "035",
    "카카오뱅크": "090",
    "케이뱅크": "089",
    "한국씨티은행": "027",
    "BOA": "060",
    "HSBC": "054",
    "JP모간": "057",
    "SC제일은행": "023",
    "하나증권": "270",
    "교보증권": "261",
    "대신증권": "267",
    "미래에셋증권": "238",
    "DB금융투자": "279",
    "유안타증권": "209",
    "메리츠증권": "287",
    "부국증권": "290",
    "삼성증권": "240",
    "신영증권": "291",
    "신한투자증권": "278",
    "NH투자증권": "247",
    "유진증권": "280",
    "키움증권": "264",
    "하이투자증권": "262",
    "한국투자": "243",
    "한화투자증권": "269",
    "KB증권": "218",
    "LS증권": "265",
    "현대차증권": "263",
    "케이프증권": "292",
    "SK증권": "266",
    "산림조합": "064",
    "중국공상은행": "062",
    "중국은행": "063",
    "중국건설은행": "067",
    "BNP파리바은행": "061",
    "한국포스증권": "294",
    "다올투자증권": "227",
    "BNK투자증권": "224",
    "카카오페이증권": "288",
    "IBK투자증권": "225",
    "토스증권": "271",
    "토스뱅크": "092",
    "상상인증권": "221"
}

# 은행명 표준화 함수
def standardize_bank_name(bank_name):
    bank_name = bank_name.lower().strip()
    if "sc제일" in bank_name or "제일" in bank_name:
        return "SC제일은행"
    elif "제일은행" in bank_name:
        return "SC제일은행"
    elif "하나" in bank_name:
        return "하나은행"
    elif "경남" in bank_name:
        return "경남은행"
    elif "광주" in bank_name:
        return "광주은행"
    elif "국민" in bank_name:
        return "국민은행"
    elif "기업" in bank_name:
        return "기업은행"
    elif "농협은행" in bank_name:
        return "농협"
    elif "NH농협" in bank_name:
        return "농협"
    elif "대구" in bank_name:
        return "iM뱅크(대구)"
    elif "대구은행" in bank_name:
        return "iM뱅크(대구)"
    elif "부산" in bank_name:
        return "부산은행"
    elif "새마을" in bank_name:
        return "새마을금고"
    elif "수협" in bank_name:
        return "수협은행"
    elif "신한" in bank_name:
        return "신한은행"
    elif "우리" in bank_name:
        return "우리은행"
    elif "전북" in bank_name:
        return "전북은행"
    elif "제주" in bank_name:
        return "제주은행"
    elif "카카오" in bank_name:
        return "카카오뱅크"
    elif "카뱅" in bank_name:
        return "카카오뱅크"
    elif "씨티" in bank_name:
        return "한국씨티은행"
    elif "토스" in bank_name:
        return "토스뱅크"
       
    # 필요한 경우 추가 은행명 표준화
    return bank_name



# 전처리 함수
def preprocess_account_info(account_info):
    # 은행명과 계좌번호 분리
    match = re.match(r'(\D+)\s*([\d\s\-]+)', account_info)
    if match:
        bank_name = standardize_bank_name(match.group(1).strip())
        account_number = re.sub(r'[\s\-]', '', match.group(2))
    else:
        bank_name = ""
        account_number = ""
    
    return bank_name, account_number

# 엑셀 파일 로드
excel_path = "C:/Users/USER/Desktop/이체매크로/이체정보.xlsx"  # 엑셀 파일 경로를 지정하세요
df = pd.read_excel(excel_path)

# 전처리된 데이터를 저장할 리스트
processed_data = []

for index, row in df.iterrows():
    product_name = row.iloc[3]  # 제품명
    customer_name = row.iloc[4]  # 이름
    account_info = row.iloc[7]  # 은행+계좌번호
    amount = row.iloc[9]  # 금액

    # 계좌정보 전처리
    bank_name, account_number = preprocess_account_info(account_info)

    # 항목 1 : 은행
    # 항목 2 : 계좌번호
    # 항목 3 : 이름.제품명
    # 항목 4 : 제품명
    name_product = f"{customer_name}{product_name}"
    processed_data.append((bank_name, account_number, name_product, product_name, amount))

# 전처리된 데이터 출력
for data in processed_data:
    print(f"은행: {data[0]}, 계좌번호: {data[1]}, 이름.제품명: {data[2]}, 제품명: {data[3]}, 금액: {data[4]}")

def type_text(element, text):
    element.clear()
    element.send_keys(text)

def type_number(text):
    for char in str(text):
        pyautogui.press(char)
        tm.sleep(0.005)

def input_transfer_info(data, index):
    bank, account_number, name_product, product_name, amount = data
    
    try:
        # 은행 선택
        bank_element = driver.find_element(By.ID, f"rcvBnkCd{index}")
        option_value = bank_options.get(bank, "")
        if option_value:
            option = bank_element.find_element(By.CSS_SELECTOR, f"option[value='{option_value}']")
            option.click()
            print(f"{name_product} 은행 선택 성공: {bank}")
        else:
            print(f"{name_product} 은행을 찾을 수 없습니다: {bank}")
        
        # 계좌번호 입력
        account_element = driver.find_element(By.ID, f"rcvAcctNo{index}")
        account_element.click()
        tm.sleep(0.1)  # 첫 번째 문자 입력 전에 짧은 지연을 추가
        type_number(account_number)
        print(f"{name_product} 계좌번호 입력 성공: {account_number}")
        
        # 금액 입력
        amount_element = driver.find_element(By.ID, f"trnsAmt{index}")
        amount_element.click()
        tm.sleep(0.1)  # 첫 번째 문자 입력 전에 짧은 지연을 추가
        type_number(int(amount))
        print(f"{name_product} 금액 입력 성공: {amount}")
        
        # 이름.제품명 입력
        name_product_element = driver.find_element(By.ID, f"wdrwPsbkMarkCtt{index}")
        type_text(name_product_element, name_product)
        print(f"{name_product} 이름.제품명 입력 성공: {name_product}")
        
        # 제품명 입력
        product_name_element = driver.find_element(By.ID, f"rcvPsbkMarkCtt{index}")
        type_text(product_name_element, product_name)
        print(f"{name_product} 제품명 입력 성공: {product_name}")

    except Exception as e:
        print(f"오류 발생 {name_product}: {e}")

# # paymAcctPw에 '5800' 입력
# def enter_password():
#     try:
#         password_element = driver.find_element(By.ID, "paymAcctPw")
#         password_element.click()
#         tm.sleep(0.1)  # 첫 번째 문자 입력 전에 짧은 지연을 추가
#         type_number("5800")
#         print("비밀번호 입력 성공")
#     except Exception as e:
#         print(f"비밀번호 입력 오류: {e}")

# 비밀번호 입력
# enter_password()

# 최대 10개의 항목 입력
for index, data in enumerate(processed_data):
    if index >= 10:
        break
    input_transfer_info(data, index)
    tm.sleep(0.5)  # 각 세트 완료 후 잠시 대기

# 나머지 코드는 필요에 따라 추가

## 1.2ver 완성

Current working directory: C:\Users\USER\Desktop\이체매크로
공동인증서 로그인(구 공인인증서)
localdisk.png을(를) 찾고 클릭합니다.
localdisk.png의 위치를 찾았습니다: (597, 367), 클릭합니다.
seagate_bt.png을(를) 찾고 클릭합니다.
seagate_bt.png의 위치를 찾았습니다: (651, 421), 클릭합니다.
localdisk_loginbt.png을(를) 찾고 클릭합니다.
localdisk_loginbt.png의 위치를 찾았습니다: (520, 742), 클릭합니다.
send_tab.png을(를) 찾고 클릭합니다.
send_tab.png의 위치를 찾았습니다: (523, 213), 클릭합니다.
은행: 카카오뱅크, 계좌번호: 3333283123277, 이름.제품명: 한상헌(박지항)왕스프, 제품명: 왕스프, 금액: 4800
은행: 농협, 계좌번호: 3560983252143, 이름.제품명: 이세은왕스프, 제품명: 왕스프, 금액: 4800
은행: 토스뱅크, 계좌번호: 100040822309, 이름.제품명: 김희정왕스프, 제품명: 왕스프, 금액: 4800
은행: 농협, 계좌번호: 111612161830, 이름.제품명: 이승현왕스프, 제품명: 왕스프, 금액: 4800
한상헌(박지항)왕스프 은행 선택 성공: 카카오뱅크
한상헌(박지항)왕스프 계좌번호 입력 성공: 3333283123277
한상헌(박지항)왕스프 금액 입력 성공: 4800
한상헌(박지항)왕스프 이름.제품명 입력 성공: 한상헌(박지항)왕스프
한상헌(박지항)왕스프 제품명 입력 성공: 왕스프
이세은왕스프 은행 선택 성공: 농협
이세은왕스프 계좌번호 입력 성공: 3560983252143
이세은왕스프 금액 입력 성공: 4800
이세은왕스프 이름.제품명 입력 성공: 이세은왕스프
이세은왕스프 제품명 입력 성공: 왕스프
김희정왕스프 은행 선택 성공: 토스뱅크
김희정왕스프 계좌번호 입력 성공: 100040822309
김희정왕스프 금액 입력 성공: 

In [4]:
맨 밑에 2개를 사용하면 됨

SyntaxError: invalid syntax (3854354703.py, line 1)

In [2]:
import subprocess
import pyautogui
import time as tm
import cv2
import numpy as np
import mss
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 작업 디렉토리 설정
os.chdir("C:/Users/USER/Desktop/이체매크로")
print("Current working directory:", os.getcwd())

# 이미지 파일 경로 설정
COORDS = {
    "localdisktab": "localdisk.png",
    "outdisktab": "seagate_bt.png",
    "commonlogin": "localdisk_loginbt.png",
    "send_tab": "send_tab.png",
    "multisend": "multisend_bt.png",
    "password4_input": "password4_input.png",
}

monitor_1 = {'top': 0, 'left': 1920, 'width': 1920, 'height': 1080}
monitor_2 = {'top': 0, 'left': 0, 'width': 1920, 'height': 1080}

def capture_screen(region):
    with mss.mss() as sct:
        screenshot = sct.grab(region)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        return img

def match_template(screen, template_path):
    template = cv2.imread(template_path, cv2.IMREAD_COLOR)
    if template is None:
        raise ValueError(f"이미지를 찾을 수 없습니다: {template_path}")

    result = cv2.matchTemplate(screen, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)
    if max_val >= 0.9:  # 일치율이 90% 이상인 경우
        return (max_loc[0] + template.shape[1] // 2, max_loc[1] + template.shape[0] // 2)
    return None

def locate_image_on_monitor(image_path, monitor):
    screen = capture_screen(monitor)
    location = match_template(screen, image_path)
    if location:
        location = (location[0] + monitor['left'], location[1] + monitor['top'])
    return location

def locate_image_on_monitors(image_path):
    location = locate_image_on_monitor(image_path, monitor_1)
    if not location:
        location = locate_image_on_monitor(image_path, monitor_2)
    return location

def locate_and_click(image_path):
    print(f"{image_path}을(를) 찾고 클릭합니다.")
    location = locate_image_on_monitors(image_path)
    if location:
        print(f"{image_path}의 위치를 찾았습니다: {location}, 클릭합니다.")
        pyautogui.click(location)
        tm.sleep(0.5)
    else:
        print(f"{image_path}의 위치를 찾을 수 없습니다.")

def scroll_down():
    pyautogui.scroll(-10000)

# Selenium 설정 및 웹페이지 열기
url = 'https://www.kebhana.com/common/login.do'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

tm.sleep(5)

print(driver.find_element(By.CSS_SELECTOR, "#certLogin").text) 
driver.find_element(By.CSS_SELECTOR, "#certLogin").click()

tm.sleep(10)

# PyAutoGUI를 이용한 인증서 로그인
def login_with_certificate():
    locate_and_click(COORDS["localdisktab"])   # 로컬디스크
    tm.sleep(1)
    locate_and_click(COORDS["outdisktab"])     # 외장하드선택
    tm.sleep(3)
    
    for _ in range(6):   # 탭 6번
        pyautogui.press('tab')
        tm.sleep(0.3)
    
    # 비밀번호 입력 (한글자씩 입력)
    password = "!gusqls120"
    for char in password:
        pyautogui.press(char)
        tm.sleep(0.2)
    
    locate_and_click(COORDS["commonlogin"])    # 공동로그인 버튼 클릭

login_with_certificate()
tm.sleep(10)

# 이체 탭 클릭
locate_and_click(COORDS["send_tab"])
tm.sleep(1)

# 다계좌 이체 버튼 클릭
for _ in range(3):   # 탭 3번
    pyautogui.press('tab')

pyautogui.press('enter')   
tm.sleep(3)


def scroll_down(amount):
    pyautogui.scroll(-amount)  # 스크롤 다운

def scroll_up(amount):
    pyautogui.scroll(amount)  # 스크롤 업

# 예시: 전체 페이지를 스크롤 다운하고 절반만 다시 스크롤 업
scroll_amount = 1000  # 페이지의 길이에 맞게 조정
scroll_down(scroll_amount)
tm.sleep(2)  # 잠시 대기
scroll_up(scroll_amount // 2)


##입금정보 입력하기 

driver.find_element(By.CSS_SELECTOR, "#rcvBnkCd0").click()


#① 공인인증서 로그인 후 다계좌이체까지 

Current working directory: C:\Users\USER\Desktop\이체매크로
공동인증서 로그인(구 공인인증서)
localdisk.png을(를) 찾고 클릭합니다.
localdisk.png의 위치를 찾았습니다: (597, 367), 클릭합니다.
seagate_bt.png을(를) 찾고 클릭합니다.
seagate_bt.png의 위치를 찾았습니다: (651, 421), 클릭합니다.
localdisk_loginbt.png을(를) 찾고 클릭합니다.
localdisk_loginbt.png의 위치를 찾았습니다: (520, 742), 클릭합니다.
send_tab.png을(를) 찾고 클릭합니다.
send_tab.png의 위치를 찾았습니다: (523, 213), 클릭합니다.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#rcvBnkCd0"}
  (Session info: chrome=126.0.6478.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x006FC1C3+27395]
	(No symbol) [0x00693DC4]
	(No symbol) [0x00591B7F]
	(No symbol) [0x005D2C65]
	(No symbol) [0x005D2D3B]
	(No symbol) [0x0060EC82]
	(No symbol) [0x005F39E4]
	(No symbol) [0x0060CB24]
	(No symbol) [0x005F3736]
	(No symbol) [0x005C7541]
	(No symbol) [0x005C80BD]
	GetHandleVerifier [0x009B3A93+2876371]
	GetHandleVerifier [0x00A07F5D+3221661]
	GetHandleVerifier [0x0077D634+556916]
	GetHandleVerifier [0x0078474C+585868]
	(No symbol) [0x0069CE04]
	(No symbol) [0x00699818]
	(No symbol) [0x006999B7]
	(No symbol) [0x0068BF0E]
	BaseThreadInitThunk [0x761A7BA9+25]
	RtlInitializeExceptionChain [0x779DC10B+107]
	RtlClearBits [0x779DC08F+191]


In [12]:
import pandas as pd
import pyautogui
import time as tm

# 엑셀 파일 읽기
excel_path = "C:/Users/USER/Desktop/이체매크로/이체정보.xlsx"
df = pd.read_excel(excel_path)

# H열 (index 7)에서 은행과 계좌번호 가져오기 (첫 행은 헤더)
account_data = df.iloc[0:, 7]

# 계좌번호만 입력하는 함수
def type_account_number(account_number):
    for char in account_number:
        pyautogui.press(char)
        tm.sleep(0.01)

# 카운트다운 함수
def countdown(seconds):
    for i in range(seconds, 0, -1):
        print(f"{i}초 남았습니다.")
        tm.sleep(1)

# 계좌번호 입력 작업
for account in account_data:
    bank, account_number = account.split()
    countdown(4)
    type_account_number(account_number)


#1.임시완성_계좌번호입력 / 4초 

4초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
4초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
4초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
4초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.


In [10]:
import pandas as pd
import pyautogui
import time as tm

# 엑셀 파일 읽기
excel_path = "C:/Users/USER/Desktop/이체매크로/이체정보.xlsx"
df = pd.read_excel(excel_path)

# J열 (index 9)에서 금액 정보 가져오기 (첫 행은 헤더)
amount_data = df.iloc[0:, 9]

# 숫자 입력하는 함수
def type_number(number):
    for char in number:
        pyautogui.press(char)
        tm.sleep(0.01)

# 카운트다운 함수
def countdown(seconds):
    for i in range(seconds, 0, -1):
        print(f"{i}초 남았습니다.")
        tm.sleep(1)

# 금액 입력 작업
for amount in amount_data:
    countdown(3)
    type_number(str(amount))



#2.임시완성_금액입력 / 3초 


3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.


In [6]:
#1-2.임시완성_금액입력 / 최초3초 1초 

import pandas as pd
import pyautogui
import time as tm

# 엑셀 파일 읽기
excel_path = "C:/Users/USER/Desktop/이체매크로/이체정보.xlsx"
df = pd.read_excel(excel_path)

# H열 (index 7)에서 은행과 계좌번호 가져오기 (첫 행은 헤더)
account_data = df.iloc[0:, 7]

# 계좌번호만 입력하는 함수
def type_account_number(account_number):
    for char in account_number:
        pyautogui.press(char)
        tm.sleep(0.01)

# 카운트다운 함수
def countdown(seconds):
    for i in range(seconds, 0, -1):
        print(f"{i}초 남았습니다.")
        tm.sleep(1)

# 계좌번호 입력 작업
first_action = True
for account in account_data:
    if first_action:
        countdown(5)  # 최초 카운트다운
        first_action = False
    
    bank, account_number = account.split()
    type_account_number(account_number)  # 계좌번호 입력
    
    x, y = pyautogui.position()  # 현재 마우스 위치
    pyautogui.moveTo(x, y + 43)  # 마우스 커서 45픽셀 이동
    pyautogui.click()  # 클릭
    
    countdown(1)  # 카운트다운 시작



#1-2.임시완성_금액입력 / 최초3초 1초 


5초 남았습니다.
4초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
1초 남았습니다.
1초 남았습니다.
1초 남았습니다.
1초 남았습니다.


In [1]:
#2-2.임시완성_금액입력 / 최초3초 1초 

import pandas as pd
import pyautogui
import time as tm

# 엑셀 파일 읽기
excel_path = "C:/Users/USER/Desktop/이체매크로/이체정보.xlsx"
df = pd.read_excel(excel_path)

# J열 (index 9)에서 금액 정보 가져오기 (첫 행은 헤더)
amount_data = df.iloc[0:, 9]

# 숫자 입력하는 함수
def type_number(number):
    for char in number:
        pyautogui.press(char)
        tm.sleep(0.01)

# 카운트다운 함수
def countdown(seconds):
    for i in range(seconds, 0, -1):
        print(f"{i}초 남았습니다.")
        tm.sleep(1)

# 금액 입력 작업
first_action = True
for amount in amount_data:
    if first_action:
        countdown(5)  # 최초 카운트다운
        first_action = False
    
    type_number(str(amount))  # 숫자 입력
    
    x, y = pyautogui.position()  # 현재 마우스 위치
    pyautogui.moveTo(x, y + 43)  # 마우스 커서 45픽셀 이동
    pyautogui.click()  # 클릭
    
    countdown(1)  # 카운트다운 시작

#2-2.임시완성_금액입력 / 최초3초 1초 

5초 남았습니다.
4초 남았습니다.
3초 남았습니다.
2초 남았습니다.
1초 남았습니다.
1초 남았습니다.
